In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pathlib
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3468, done.
remote: Counting objects: 100% (3468/3468), done.
remote: Compressing objects: 100% (2785/2785), done.
remote: Total 3468 (delta 1043), reused 1525 (delta 634), pack-reused 0
Receiving objects: 100% (3468/3468), 34.30 MiB | 18.02 MiB/s, done.
Resolving deltas: 100% (1043/1043), done.


In [3]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1686356 sha256=2b42a84d9d6807cc5ede2e27af01cc4cd074146cf300dd643488a9080ba31b5f
  Stored in directory: /tmp/pip-ephem-wheel-cache-xala3sfa/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=cf30b563345e1401a4a08852f4103994d2cfa06f24d613eed37d719a553cc067
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=0b3cea9d95d1b40b1a67b7d4d0871376f727ab17212553e1a10a6bd0c3eb81d8
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=f26b9ee54b27133f5d21b83a1474e203a7ee58896652411c74

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [15]:
import matplotlib
import matplotlib.pyplot as plt
import time
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.1)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

In [5]:
NUM_CLASSES = 8

category_index = {1: {'id': 1, 'name': 'bone'},
                  2: {'id': 2, 'name': 'abdomen'},
                  3: {'id': 3, 'name': 'mediastinum'},
                  4: {'id': 4, 'name': 'liver'},
                  5: {'id': 5, 'name': 'lung'},
                  6: {'id': 6, 'name': 'kidney'},
                  7: {'id': 7, 'name': 'soft_tissue'},
                  8: {'id': 8, 'name': 'pelvis'}}

def _parse_function(example_proto):
    feature_description = {
        'image/height': tf.io.FixedLenFeature((), tf.int64, default_value=1),
        'image/width': tf.io.FixedLenFeature((), tf.int64, default_value=1),
        'image/win_min': tf.io.FixedLenFeature((), tf.float32, default_value=-1024.),
        'image/win_max': tf.io.FixedLenFeature((), tf.float32, default_value=3071.), 
        'image/filename': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/source_id': tf.io.FixedLenFeature((), tf.string, default_value=''),    
        'image/encoded': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/format': tf.io.FixedLenFeature((), tf.string, default_value='jpeg'),
        # Object boxes and classes.
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }  
    return tf.io.parse_single_example(example_proto, feature_description)

def GetImageLabelBoxTensors(image_features, batch_size):
    train_image_tensors = []
    gt_box_tensors = []
    gt_classes_one_hot_tensors = [] 
    num_classes = NUM_CLASSES
    label_id_offset = 1
    for i in range(batch_size):
#       print('i = {}'.format(i))
        image = tf.io.decode_png(image_features['image/encoded'][i], channels=3)
        image = tf.cast(image, tf.float32)
        image = tf.expand_dims(image, axis=0)
        xmins = tf.sparse.to_dense(image_features['image/object/bbox/xmin'])[i:i+1]
        xmaxs = tf.sparse.to_dense(image_features['image/object/bbox/xmax'])[i:i+1]
        ymins = tf.sparse.to_dense(image_features['image/object/bbox/ymin'])[i:i+1]
        ymaxs = tf.sparse.to_dense(image_features['image/object/bbox/ymax'])[i:i+1]
        bbox = tf.concat([ymins, xmins, ymaxs, xmaxs], 0)        
        bbox = tf.transpose(bbox)
        labels_np = tf.sparse.to_dense(image_features['image/object/class/label'])[i].numpy()
        zero_args = np.argwhere(labels_np == 0)
        if zero_args.size > 0:
            num_bbox = zero_args[0][0]
            labels_np = labels_np[:num_bbox]
            bbox = bbox[:num_bbox] 
#            print('labels = {}'.format(labels_np))
        zero_indexed_groundtruth_classes = tf.convert_to_tensor(labels_np - label_id_offset)
        gt_classes_one_hot_tensors.append(tf.one_hot(zero_indexed_groundtruth_classes, num_classes))
        train_image_tensors.append(image)
        gt_box_tensors.append(bbox)    
    return train_image_tensors, gt_box_tensors, gt_classes_one_hot_tensors

# Again, uncomment this decorator if you want to run inference eagerly
@tf.function
def detect(input_tensor):

    """Run detection on an input image.
    Args:
    input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
    Note that height and width can be anything since the image will be
    immediately resized according to the needs of the model within this
    function.

    Returns:
    A dict containing 3 Tensors (`detection_boxes`, `detection_classes`,
    and `detection_scores`).
    """
    preprocessed_image, shapes = detection_model.preprocess(input_tensor)
    prediction_dict = detection_model.predict(preprocessed_image, shapes)
    return detection_model.postprocess(prediction_dict, shapes)

def get_latest_ckpt(dir):
    search = dir + '/*.index'
    ckpts = glob.glob(search)
    lastest_ckpt = ''
    max_idx = -1
    for item in ckpts:
        item1 = item[:item.index('.')]
        idx = int(item1[item1.index('-')+1:])
        #print(item1, idx)
        if (idx > max_idx):
            max_idx = idx
            lastest_ckpt = item1
    return lastest_ckpt

# bbox[ymin, xmin, ymax, xmax]
def get_iou(bbox1, bbox2):
    if (bbox1[1] > bbox2[3] or bbox2[1] > bbox1[3] or bbox1[0] > bbox2[2] or bbox2[0] > bbox1[2]):
        return 0.0
    overlap = ((min(bbox1[3], bbox2[3]) - max(bbox1[1], bbox2[1])) * (min(bbox1[2], bbox2[2]) - max(bbox1[0], bbox2[0])))
    union = ((bbox1[3] - bbox1[1]) * (bbox1[2] - bbox1[0]) + (bbox2[3] - bbox2[1]) * (bbox2[2] - bbox2[0]) - overlap)
#    print('overlap = {}, union = {}, ret = {}'.format(overlap, union, overlap/union))
    return overlap / union

def box1in2(bbox1, bbox2):
    cx = (bbox1[1] + bbox1[3]) / 2
    cy = (bbox1[0] + bbox1[2]) / 2
    return (cy > bbox2[0] and cy < bbox2[2] and cx > bbox2[1] and cx < bbox2[3])

def merge_markers(in_markers, in_scores, in_labels, low_score=0.1, iou_thrd=0.4):
    delete_id = []
    for i in range(len(in_scores)):
        if (in_scores[i] <= low_score):
            delete_id.append(i)
    in_markers = np.delete(in_markers, delete_id, 0)
    in_scores = np.delete(in_scores, delete_id, 0)
    in_labels = np.delete(in_labels, delete_id, 0)
    delete_id = []
    for i in range(len(in_scores)):
        for j in range(i+1, len(in_scores)):
            iou = get_iou(in_markers[i], in_markers[j])
            if (iou >= iou_thrd):
                if (in_scores[i] > in_scores[j]):
                    delete_id.append(j)
                else:
                    delete_id.append(i)    
    out_markers = np.delete(in_markers, delete_id, 0)
    out_scores = np.delete(in_scores, delete_id, 0)
    out_labels = np.delete(in_labels, delete_id, 0)       
    return out_markers, out_scores, out_labels    

In [12]:
def score_froc(testset):
    #froc parameters
    hit_iou = 0.25
    min_score = 0.1
    num_images = 0.0
    num_lesions = 0.0
    froc_fp = np.zeros(1001)
    froc_sen = np.zeros(1001)
    loose_score = False
    label_id_offset = 1
    for image_features in testset:
#        filename = image_features['image/filename'].numpy()[0].decode("utf-8") 
        image_tensors, gt_boxes_list, gt_classes_list = GetImageLabelBoxTensors(image_features, 1)
        detections = detect(image_tensors[0])
                
        det_boxes = detections['detection_boxes'][0].numpy()
        det_labels = detections['detection_classes'][0].numpy().astype(np.uint32) + label_id_offset
        det_scores = detections['detection_scores'][0].numpy()
        out_boxes, out_scores, out_labels = merge_markers(det_boxes, det_scores, det_labels, low_score=min_score, iou_thrd=0.4)
        gt_boxes = gt_boxes_list[0].numpy()
        gt_labels = gt_classes_list
        num_det = len(out_scores)
        num_tru = len(gt_boxes)
        
        b_hit = False
        hits_score = []
        good_det_id = []
        for tru_id in range(num_tru):
            max_hit_score = 0.0
            for det_id in range(num_det):
                if loose_score:
                    b_hit = box1in2(out_boxes[det_id], gt_boxes[tru_id])
                else:
                    iou = get_iou(gt_boxes[tru_id], out_boxes[det_id])
                    b_hit = (iou >= hit_iou)
#                print('tru_id = {}, det_id = {}, iou = {}'.format(tru_id, det_id, iou))
                if b_hit:
                    max_hit_score = max(max_hit_score, out_scores[det_id])
                    good_det_id.append(det_id)
            hits_score.append(max_hit_score)
        hits_score = np.array(hits_score)
        good_det_id = np.unique(good_det_id).astype(np.int)
        fps_score = np.delete(out_scores, good_det_id, None)
        num_images += 1.0
        num_lesions += float(num_tru)
        print('{}: tru_boxes = {}, det_box = {}, hits_score = {}, fps_score = {}'.format(num_images, num_tru, num_det, len(hits_score), len(fps_score)))
            
        for t in range(0, 1001):
            th = t / 1000
            hit = len(hits_score[hits_score > th])
            fp = len(fps_score[fps_score > th])
            froc_sen[t] += hit
            froc_fp[t] += fp
        
        if (num_images <= 10):
            image_np = (image_tensors[0].numpy())[0]
            image_np = image_np.astype(np.uint8)
            gt_labels = np.ones_like(gt_classes_list[0].numpy().reshape((-1,))) * 99
            gt_scores = np.ones_like(gt_labels)
            gt_labels = gt_labels.astype(det_labels.dtype)
            out_boxes = np.concatenate((out_boxes, gt_boxes), axis=0)
            out_labels = np.concatenate((out_labels, gt_labels), axis=0)
            out_scores = np.concatenate((out_scores, gt_scores), axis=0)
            plot_detections(image_np, out_boxes, out_labels, out_scores, category_index, figsize=(15, 20), image_name="/content/drive/MyDrive/demo_" + ('%02d' % num_images) + ".jpg")
            
    froc_fp /= num_images
    froc_sen /= num_lesions
    with open(('/content/drive/MyDrive/froc.txt'), 'w') as fp:
        fp.write('number of lesion = {}, number of iamges = {}\n'.format(int(num_lesions), int(num_images)))
        for i in range(0, 1001):
#            print('{:.4f}  {:.4f}'.format(froc_fp[i], froc_sen[i]))
            fp.write('{:.4f}  {:.4f}\n'.format(froc_fp[i], froc_sen[i]))
        print('number of lesion = {}, number of iamges = {}'.format(int(num_lesions), int(num_images)))            
    plt.figure()
    plt.grid(b=True)
    plt.plot(froc_fp, froc_sen, 'b')
    plt.savefig('/content/drive/MyDrive/froc.png')
    print('high sensitivity = {}, total fp = {}'.format(froc_sen[0], froc_fp[0]))    
    return froc_fp, froc_sen

In [13]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

--2022-03-21 14:53:45--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.203.128, 2607:f8b0:400e:c09::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.203.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz.1’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M   200MB/s    in 1.2s    

2022-03-21 14:53:46 (200 MB/s) - ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz.1’ saved [244817203/244817203]

mv: cannot move 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint' to 'models/research/object_detection/test_data/checkpoint': Directory not empty


In [8]:
tf.keras.backend.clear_session()

print('Building model and restoring weights for fine-tuning...', flush=True)
num_classes = NUM_CLASSES
pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
checkpoint_path = get_latest_ckpt('/content/drive/MyDrive/my_model')

# Load pipeline config and build a detection model.
#
# Since we are working off of a COCO architecture which predicts 90
# class slots by default, we override the `num_classes` field here to be just
# one (for our new rubber ducky class).
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True
detection_model = model_builder.build(model_config=model_config, is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
if (len(checkpoint_path) > 0):
    ckpt.restore(checkpoint_path).expect_partial()

# Run model through a dummy image so that variables are created
image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
prediction_dict = detection_model.predict(image, shapes)
_ = detection_model.postprocess(prediction_dict, shapes)
print('Weights restored!')

Building model and restoring weights for fine-tuning...
Weights restored!


In [ ]:
batch_size = 1
eval_tfr_path = ['/content/drive/MyDrive/data1/val_deeplesion.record-00028-of-00030']
testset = tf.data.TFRecordDataset(eval_tfr_path)
testset = testset.map(_parse_function)
testset = testset.batch(batch_size) 

tf.keras.backend.clear_session()
_, _ = score_froc(testset)